In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error



In [100]:
raw_data = pd.read_csv('data/Ames_Housing_Price_Data.csv')

#remove the 3 neighborhoods
raw_data = raw_data[raw_data['Neighborhood'] != 'Landmrk']
raw_data = raw_data[raw_data['Neighborhood'] != 'GrnHill']
raw_data = raw_data[raw_data['Neighborhood'] != 'Greens']

In [101]:
basementDF = raw_data[['PID','SalePrice','BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF']]
basementDF.head()

,PID,SalePrice,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF
0,909176150,126000,TA,TA,No,Rec,238.0,Unf,0.0,618.0,856.0
1,905476230,139500,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0
2,911128020,124900,TA,TA,No,ALQ,737.0,Unf,0.0,100.0,837.0
3,535377150,114000,Fa,TA,No,Unf,0.0,Unf,0.0,405.0,405.0
4,534177230,227000,Gd,TA,No,GLQ,643.0,Unf,0.0,167.0,810.0


In [102]:
basementDF.BsmtQual = basementDF['BsmtQual'].fillna(0)
basementDF.BsmtQual = basementDF.BsmtQual.replace(to_replace='Ex', value=5)
basementDF.BsmtQual = basementDF.BsmtQual.replace(to_replace='Gd', value=4)
basementDF.BsmtQual = basementDF.BsmtQual.replace(to_replace='TA', value=3)
basementDF.BsmtQual = basementDF.BsmtQual.replace(to_replace='Fa', value=2)
basementDF.BsmtQual = basementDF.BsmtQual.replace(to_replace='Po', value=1)

basementDF.BsmtCond = basementDF['BsmtCond'].fillna(0)
basementDF.BsmtCond = basementDF.BsmtCond.replace(to_replace='Ex', value=5)
basementDF.BsmtCond = basementDF.BsmtCond.replace(to_replace='Gd', value=4)
basementDF.BsmtCond = basementDF.BsmtCond.replace(to_replace='TA', value=3)
basementDF.BsmtCond = basementDF.BsmtCond.replace(to_replace='Fa', value=2)
basementDF.BsmtCond = basementDF.BsmtCond.replace(to_replace='Po', value=1)

basementDF.BsmtExposure = basementDF['BsmtExposure'].fillna(0)
basementDF.BsmtExposure = basementDF.BsmtExposure.replace(to_replace='Gd', value=4)
basementDF.BsmtExposure = basementDF.BsmtExposure.replace(to_replace='Av', value=3)
basementDF.BsmtExposure = basementDF.BsmtExposure.replace(to_replace='Mn', value=2)
basementDF.BsmtExposure = basementDF.BsmtExposure.replace(to_replace='No', value=1)


basementDF.BsmtFinType1 = basementDF['BsmtFinType1'].fillna(0)
basementDF.BsmtFinType1 = basementDF.BsmtFinType1.replace(to_replace='GLQ', value=6)
basementDF.BsmtFinType1 = basementDF.BsmtFinType1.replace(to_replace='ALQ', value=5)
basementDF.BsmtFinType1 = basementDF.BsmtFinType1.replace(to_replace='BLQ', value=4)
basementDF.BsmtFinType1 = basementDF.BsmtFinType1.replace(to_replace='Rec', value=3)
basementDF.BsmtFinType1 = basementDF.BsmtFinType1.replace(to_replace='LwQ', value=2)
basementDF.BsmtFinType1 = basementDF.BsmtFinType1.replace(to_replace='Unf', value=1)

basementDF.BsmtFinType2 = basementDF['BsmtFinType2'].fillna(0)
basementDF.BsmtFinType2 = basementDF.BsmtFinType2.replace(to_replace='GLQ', value=6)
basementDF.BsmtFinType2 = basementDF.BsmtFinType2.replace(to_replace='ALQ', value=5)
basementDF.BsmtFinType2 = basementDF.BsmtFinType2.replace(to_replace='BLQ', value=4)
basementDF.BsmtFinType2 = basementDF.BsmtFinType2.replace(to_replace='Rec', value=3)
basementDF.BsmtFinType2 = basementDF.BsmtFinType2.replace(to_replace='LwQ', value=2)
basementDF.BsmtFinType2 = basementDF.BsmtFinType2.replace(to_replace='Unf', value=1)

In [103]:
basementDF = basementDF[basementDF.BsmtFinSF1.notna()] # 1 obs dropped

In [104]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

X = basementDF.copy().iloc[:,2:]
X =  add_constant(X)

pd.Series([variance_inflation_factor(X.values, i) for i in range(X.shape[1])], index=X.columns)

const           29.350531
BsmtQual         2.358155
BsmtCond         1.780682
BsmtExposure     1.392391
BsmtFinType1     2.395038
BsmtFinSF1            inf
BsmtFinType2     3.090069
BsmtFinSF2            inf
BsmtUnfSF             inf
TotalBsmtSF           inf
dtype: float64

In [105]:
basementDF = basementDF.drop(['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF'], axis=1, inplace=False)

In [36]:
X['Condition_Square'] = X['BsmtCond']**2
X['Condition_cube'] = X['BsmtCond']**3
X['Exposure_Square'] = X['BsmtExposure']**2
X['Exposure_cube'] = X['BsmtExposure']**3
X['Exposure_quartic'] = X['BsmtExposure']**4
X['type1_sq'] = X['BsmtFinType1']**2
X['type1_cube'] = X['BsmtFinType1']**3
X['type1_quartic'] = X['BsmtFinType1']**4
X['type2_sq'] = X['BsmtFinType2']**2
X['type2_cube'] = X['BsmtFinType2']**3
X['type2_quartic'] = X['BsmtFinType2']**4


In [87]:
X1 = basementDF.copy().iloc[:,2:]
X1['Condition_Square'] = X1['BsmtCond']**2
X1['Condition_cube'] = X1['BsmtCond']**3

y = np.log(basementDF['SalePrice'])

score = []
for i in range(100):
    Xtrain, Xtest, ytrain, ytest = train_test_split(X1, y, shuffle=True, test_size=0.15)
    ols = LinearRegression()
    ols.fit(Xtrain, ytrain)
    adj_R_square = 1 - (1-ols.score(Xtest, ytest))*(len(ytest)-1)/(len(ytest)-Xtest.shape[1]-1)
    score.append(adj_R_square)

print(np.average(score))

0.45977358407073265


In [88]:
X2 = basementDF.copy().iloc[:,2:]
X2['Exposure_Square'] = X2['BsmtExposure']**2
X2['Exposure_cube'] = X2['BsmtExposure']**3
X2['Exposure_quartic'] = X2['BsmtExposure']**4

y = np.log(basementDF['SalePrice'])

score = []
for i in range(100):
    Xtrain, Xtest, ytrain, ytest = train_test_split(X2, y, shuffle=True, test_size=0.15)
    ols = LinearRegression()
    ols.fit(Xtrain, ytrain)
    adj_R_square = 1 - (1-ols.score(Xtest, ytest))*(len(ytest)-1)/(len(ytest)-Xtest.shape[1]-1)
    score.append(adj_R_square)

print(np.average(score))

0.46651444314384505


In [89]:
X3 = basementDF.copy().iloc[:,2:]
X3['type1_sq'] = X3['BsmtFinType1']**2
X3['type1_cube'] = X3['BsmtFinType1']**3
X3['type1_quartic'] = X3['BsmtFinType1']**4

y = np.log(basementDF['SalePrice'])

score = []
for i in range(100):
    Xtrain, Xtest, ytrain, ytest = train_test_split(X3, y, shuffle=True, test_size=0.15)
    ols = LinearRegression()
    ols.fit(Xtrain, ytrain)
    adj_R_square = 1 - (1-ols.score(Xtest, ytest))*(len(ytest)-1)/(len(ytest)-Xtest.shape[1]-1)
    score.append(adj_R_square)

print(np.average(score))

0.45960711318627484


In [90]:
X4 = basementDF.copy().iloc[:,2:]
X4['Condition_Square'] = X4['BsmtCond']**2
X4['Condition_cube'] = X4['BsmtCond']**3
X4['Exposure_Square'] = X4['BsmtExposure']**2
X4['Exposure_cube'] = X4['BsmtExposure']**3
X4['Exposure_quartic'] = X4['BsmtExposure']**4

y = np.log(basementDF['SalePrice'])

score = []
for i in range(100):
    Xtrain, Xtest, ytrain, ytest = train_test_split(X4, y, shuffle=True, test_size=0.15)
    ols = LinearRegression()
    ols.fit(Xtrain, ytrain)
    adj_R_square = 1 - (1-ols.score(Xtest, ytest))*(len(ytest)-1)/(len(ytest)-Xtest.shape[1]-1)
    score.append(adj_R_square)

print(np.average(score))

0.4630009562536208


In [91]:
X5 = basementDF.copy().iloc[:,2:]
y = np.log(basementDF['SalePrice'])

score = []
for i in range(100):
    Xtrain, Xtest, ytrain, ytest = train_test_split(X5, y, shuffle=True, test_size=0.15)
    ols = LinearRegression()
    ols.fit(Xtrain, ytrain)
    adj_R_square = 1 - (1-ols.score(Xtest, ytest))*(len(ytest)-1)/(len(ytest)-Xtest.shape[1]-1)
    score.append(adj_R_square)

print(np.average(score))


0.42297722214774064


In [92]:
X6 = basementDF.copy().iloc[:,2:]
X6['Condition_Square'] = X6['BsmtCond']**2
X6['Condition_cube'] = X6['BsmtCond']**3
X6['type1_sq'] = X6['BsmtFinType1']**2
X6['type1_cube'] = X6['BsmtFinType1']**3
X6['type1_quartic'] = X6['BsmtFinType1']**4

y = np.log(basementDF['SalePrice'])

score = []
for i in range(100):
    Xtrain, Xtest, ytrain, ytest = train_test_split(X6, y, shuffle=True, test_size=0.15)
    ols = LinearRegression()
    ols.fit(Xtrain, ytrain)
    adj_R_square = 1 - (1-ols.score(Xtest, ytest))*(len(ytest)-1)/(len(ytest)-Xtest.shape[1]-1)
    score.append(adj_R_square)

print(np.average(score))

0.46089264789177947


In [93]:
X7 = basementDF.copy().iloc[:,2:]
X7['Exposure_Square'] = X7['BsmtExposure']**2
X7['Exposure_cube'] = X7['BsmtExposure']**3
X7['Exposure_quartic'] = X7['BsmtExposure']**4
X7['type1_sq'] = X7['BsmtFinType1']**2
X7['type1_cube'] = X7['BsmtFinType1']**3
X7['type1_quartic'] = X7['BsmtFinType1']**4

y = np.log(basementDF['SalePrice'])

score = []
for i in range(100):
    Xtrain, Xtest, ytrain, ytest = train_test_split(X7, y, shuffle=True, test_size=0.15)
    ols = LinearRegression()
    ols.fit(Xtrain, ytrain)
    adj_R_square = 1 - (1-ols.score(Xtest, ytest))*(len(ytest)-1)/(len(ytest)-Xtest.shape[1]-1)
    score.append(adj_R_square)

print(np.average(score))

0.45356642429314903


In [94]:
X8 = basementDF.copy().iloc[:,2:]
X8['type2_sq'] = X8['BsmtFinType2']**2
X8['type2_cube'] = X8['BsmtFinType2']**3
X8['type2_quartic'] = X8['BsmtFinType2']**4

y = np.log(basementDF['SalePrice'])

score = []
for i in range(100):
    Xtrain, Xtest, ytrain, ytest = train_test_split(X8, y, shuffle=True, test_size=0.15)
    ols = LinearRegression()
    ols.fit(Xtrain, ytrain)
    adj_R_square = 1 - (1-ols.score(Xtest, ytest))*(len(ytest)-1)/(len(ytest)-Xtest.shape[1]-1)
    score.append(adj_R_square)

print(np.average(score))

0.4623754910861417


In [95]:
X9 = basementDF.copy().iloc[:,2:]
X9['type1_sq'] = X9['BsmtFinType1']**2
X9['type1_cube'] = X9['BsmtFinType1']**3
X9['type1_quartic'] = X9['BsmtFinType1']**4
X9['type2_sq'] = X9['BsmtFinType2']**2
X9['type2_cube'] = X9['BsmtFinType2']**3
X9['type2_quartic'] = X9['BsmtFinType2']**4

y = np.log(basementDF['SalePrice'])

score = []
for i in range(100):
    Xtrain, Xtest, ytrain, ytest = train_test_split(X9, y, shuffle=True, test_size=0.15)
    ols = LinearRegression()
    ols.fit(Xtrain, ytrain)
    adj_R_square = 1 - (1-ols.score(Xtest, ytest))*(len(ytest)-1)/(len(ytest)-Xtest.shape[1]-1)
    score.append(adj_R_square)

print(np.average(score))

0.46014287278109345


In [96]:
X10 = basementDF.copy().iloc[:,2:]
X10['Condition_Square'] = X10['BsmtCond']**2
X10['Condition_cube'] = X10['BsmtCond']**3
X10['Exposure_Square'] = X10['BsmtExposure']**2
X10['Exposure_cube'] = X10['BsmtExposure']**3
X10['Exposure_quartic'] = X10['BsmtExposure']**4
X10['type1_sq'] = X10['BsmtFinType1']**2
X10['type1_cube'] = X10['BsmtFinType1']**3
X10['type1_quartic'] = X10['BsmtFinType1']**4
X10['type2_sq'] = X10['BsmtFinType2']**2
X10['type2_cube'] = X10['BsmtFinType2']**3
X10['type2_quartic'] = X10['BsmtFinType2']**4

y = np.log(basementDF['SalePrice'])

score = []
for i in range(100):
    Xtrain, Xtest, ytrain, ytest = train_test_split(X9, y, shuffle=True, test_size=0.15)
    ols = LinearRegression()
    ols.fit(Xtrain, ytrain)
    adj_R_square = 1 - (1-ols.score(Xtest, ytest))*(len(ytest)-1)/(len(ytest)-Xtest.shape[1]-1)
    score.append(adj_R_square)

print(np.average(score))

0.45735187797143584


In [107]:
X2

,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Exposure_Square,Exposure_cube,Exposure_quartic
0,3,3,1,3,1,1,1,1
1,4,3,2,6,5,4,8,16
2,3,3,1,5,1,1,1,1
3,2,3,1,1,1,1,1,1
4,4,3,1,6,1,1,1,1
...,...,...,...,...,...,...,...,...
2575,3,3,1,1,1,1,1,1
2576,0,0,0,0,0,0,0,0
2577,3,3,1,4,1,1,1,1
2578,4,3,1,6,1,1,1,1


According to the adjusted R_Square value, the best equation is:

$log(\hat{y}) = \beta_{0} + \beta_{1}BsmtQual + \beta_{2}BsmtCond + \beta_{3}BsmtExposure + \beta_{4}BsmtExposure^{2} + \beta_{5}BsmtExposure^{3} + \beta_{6}BsmtExposure^{4} + \beta_{7}BsmtFinType1 + \beta_{8}BsmtFinType2$

